# Imports & private packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Private packages via Google Drive

In [ ]:
cd "/content/drive/Othercomputers/PC/Masterarbeit/5 Programmcodes/packages/sgte"

/content/drive/Othercomputers/PC/Masterarbeit/5 Programmcodes/packages/sgte


In [ ]:
!pip install -e .

Obtaining file:///content/drive/Othercomputers/PC/Masterarbeit/5%20Programmcodes/packages/sgte
  Attempting uninstall: sgte
    Found existing installation: SGTE 0.1.0
    Can't uninstall 'SGTE'. No files were found to uninstall.
  Running setup.py develop for sgte


In [ ]:
cd "/content/drive/Othercomputers/PC/Masterarbeit/5 Programmcodes/packages/thermoclassifier"

/content/drive/Othercomputers/PC/Masterarbeit/5 Programmcodes/packages/thermoclassifier


In [ ]:
!pip install -e .

Obtaining file:///content/drive/Othercomputers/PC/Masterarbeit/5%20Programmcodes/packages/thermoclassifier
  Attempting uninstall: thermoclassifier
    Found existing installation: ThermoClassifier 0.1.0
    Can't uninstall 'ThermoClassifier'. No files were found to uninstall.
  Running setup.py develop for thermoclassifier


## Imports

In [ ]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
from thermoclassifier.dataset.dataset_creator import DatasetCreator
from thermoclassifier.elements.net import ElementClassifier

# Training routine

In [ ]:
def epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len, measurement='G'):
    epoch_losses = np.zeros([len(train_loader), ])
    correct = 0
    incorrect = 0
    
    for i, d in enumerate(train_loader):
        # Scale inputs and get predictions
        inp = d[:, :, :-1]#.squeeze(1)
        inp[:, :, 0] /= 1000
        predictions = net(inp.float().to(device))
        
        # Each batch consists of measurement batches, where seq_len measurements are put into one batch. In such a 
        # measurement batch, every measurement has the same label as it needs to be from the same element an phase. 
        # This leads to a target array where the target class is contained for each measurement in the measurement 
        # batch. With this, CrossEntropyLoss would not work as the predictions are made for the whole measurement 
        # batch and CEL therefore expects only on class label per measurement batch. Therefore, only the first 
        # element of the last dimension of d is considered as target (all the entries in the last dimension are the 
        # same anyways so it could be any entry)
        targets = d[:, :, -1][:, 0].long().to(device)

        correct += (predictions.argmax(dim=-1) == targets).sum().item()
        incorrect += len(targets) - (predictions.argmax(dim=-1) == targets).sum().item()
        
        # Calculate the loss
        loss = loss_func(predictions, targets)
        epoch_losses[i] = loss
        
        # Backward step
        net.zero_grad()
        loss.backward()
        optimizer.step()
    
    accuracy = correct/(correct + incorrect)
    print('Epoch accuracy: ', accuracy)
    return epoch_losses.mean()

In [ ]:
# Training and testing routines

def train(net, train_loader, test_loader, batch_size, seq_len, measurement='G'):
    # Hyperparameters
    nr_epochs = 10000
    lr = 0.005
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(net.parameters(), lr=lr)
    
    losses = np.zeros([nr_epochs, ])

    best_loss = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len, measurement=measurement)
    best_net = net
    
    for i in range(nr_epochs):
        losses[i] = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len, measurement=measurement)
        if losses[i] < best_loss:
            best_net = net
            best_loss = losses[i]

        if i % 10 == 0:
            print(losses[i])
            test(best_net, test_loader)
    
    return best_net

def test(net, test_loader): 
    correct = 0
    incorrect = 0
    
    for d in test_loader:
        inp = d[:, :, :-1]#.squeeze(1)
        inp[:, :, 0] /= 1000
        predictions = net(inp.float().to(device)).squeeze()#.argmax()
        targets = d[:, :, -1][:, 0].long().to(device)
        
        correct += (predictions.argmax(dim=-1) == targets).sum().item()
        incorrect += len(targets) - (predictions.argmax(dim=-1) == targets).sum().item()

    accuracy = correct/(correct + incorrect)
    print('Test accuracy: ', accuracy)
    
    return accuracy

# Run

## Network definition

In [ ]:
# Hyperparameters
seq_len = 1
measurement = 'C'
batch_size = 128

In [ ]:
def create_data_loaders(elements, splits, seq_len, validation=False, measurement='G', stable_only=True, batch_size=128, shuffle=True):
    dc = DatasetCreator(elements=elements,  splits=splits, validation=validation, seq_len=seq_len, measurement=measurement, stable_only=True)
    train_dataset, test_dataset, val_dataset = dc.get_datasets()

    # Create the DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=shuffle)
    if val_dataset:
        val_loader = DataLoader(val_dataset)
    else:
        val_loader = None

    return train_loader, test_loader, val_loader

In [ ]:
# Create the network
t = True
net = ElementClassifier(train, in_features=seq_len * 2, hidden_size_linear=32)

# Check if cuda is available and send net to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = net.to(device)

## Train

In [ ]:
train_loader, test_loader, val_loader = create_data_loaders(None, (0.8, 0.2), seq_len=seq_len, measurement=measurement)

best_net = train(net, train_loader, test_loader, batch_size, seq_len, measurement)
test(best_net, test_loader)

In [ ]:
test(best_net, test_loader)

Test accuracy:  0.1513157894736842


0.1513157894736842

In [ ]:
torch.save(best_net, 'ElementClassifier.pth')